# Markov Models Text Generation

This notebook introduces Markov chains for text generation. We'll build a simple
text generator that learns patterns from input text and generates new text with
similar statistical properties.

In [25]:
# First, let's import our required libraries
from collections import defaultdict
import random

## Building the Markov Chain

A Markov chain represents sequences of states where the probability of each state
depends only on the previous state(s). In our case, each state will be a sequence
of words, and we'll predict the next word based on this sequence.

In [26]:
def build_markov_chain(text, order=2):
    """
    Build a Markov chain from input text.

    Args:
        text (str): Input text to learn from
        order (int): Number of words to use as state (context)

    Returns:
        dict: Mapping from state tuples to lists of possible next words
    """
    chain = defaultdict(list)
    words = text.split()

    for i in range(len(words) - order):
        # Create state tuple from current words
        state = tuple(words[i:i+order])
        # Get the next word
        next_word = words[i+order]
        # Add to chain
        chain[state].append(next_word)

    return chain

## Generating Text

Now we'll use our Markov chain to generate new text. We'll randomly select from
the possible next words at each step.

In [27]:
def generate_text(chain, num_words=30):
    """
    Generate new text using the Markov chain.

    Args:
        chain (dict): Markov chain mapping states to possible next words
        order (int): Length of state tuples
        num_words (int): Number of words to generate

    Returns:
        str: Generated text
    """
    # Start with a random state from the chain
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            next_word = random.choice(chain[state])
            words.append(next_word)
        else:
            break

    return ' '.join(words)

## Part 3: Basic Example

Let's try our text generator with a simple example.

In [28]:
# Sample text
sample_text = """
The quick brown fox jumps over the lazy dog.
A quick brown dog jumps over the lazy fox.
The lazy fox sleeps while the quick brown dog watches.
"""

# Build the chain
chain = build_markov_chain(sample_text)

# Examine the chain
for state, words in chain.items():
    print(f"{' '.join(state)} -> {words}")

The quick -> ['brown']
quick brown -> ['fox', 'dog', 'dog']
brown fox -> ['jumps']
fox jumps -> ['over']
jumps over -> ['the', 'the']
over the -> ['lazy', 'lazy']
the lazy -> ['dog.', 'fox.']
lazy dog. -> ['A']
dog. A -> ['quick']
A quick -> ['brown']
brown dog -> ['jumps', 'watches.']
dog jumps -> ['over']
lazy fox. -> ['The']
fox. The -> ['lazy']
The lazy -> ['fox']
lazy fox -> ['sleeps']
fox sleeps -> ['while']
sleeps while -> ['the']
while the -> ['quick']
the quick -> ['brown']


In [29]:
# Generate some text
print("Generated text:")
print(generate_text(chain))

Generated text:
The lazy fox sleeps while the quick brown dog watches.


## Student Tasks

1. Basic Implementation:
   - Try changing the order parameter in build_markov_chain
   - What happens with order=1 vs order=3?

In [30]:
# Task 1: Experiment with different orders
print("\nOrder 1:")
chain1 = build_markov_chain(sample_text, order=1)
print(generate_text(chain1))

print("\nOrder 3:")
chain3 = build_markov_chain(sample_text, order=3)
print(generate_text(chain3))


Order 1:
the quick brown dog jumps over the lazy fox. The quick brown fox jumps over the quick brown dog watches.

Order 3:
dog. A quick brown dog jumps over the lazy dog. A quick brown dog jumps over the lazy fox. The lazy fox sleeps while the quick brown dog watches.


2. Use Your Own Text:
   Below, try using a different text source. You could use:
   - Song lyrics
   - Book excerpts
   - Movie quotes

In [31]:
# Task 2: Add your own text here
your_text = """
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions

Already passed, you don't need no approval
Good everywhere, don't worry 'bout no refusal
Second to none, you got the upper hand now
Don't need a sponsor, nope, you're the brand now
You're my rock, my Colorado
Got that ring, just like Toronto
Love you now, a little more tomorrow
This how I feel, act like you know that you are
"""

In [48]:
clean_text = your_text.lower().replace("\n", " ")

chain = build_markov_chain(clean_text, order=2)

print(generate_text_with_temperature(chain, temperature=0.8, num_words=30))

kitchen cookin' up, got your own bread heart full of equity, you're an asset make sure that you are


3. Advanced Implementation:
   Add temperature-based sampling to control randomness

In [33]:
def generate_text_with_temperature(chain, temperature=1.0, num_words=30):
    """
    Generate text with temperature-based sampling.
    Lower temperature = more conservative/predictable
    Higher temperature = more random/creative

    Args:
        chain (dict): Markov chain
        temperature (float): Controls randomness (0.1 to 2.0 recommended)
        order (int): Length of state tuples
        num_words (int): Number of words to generate
    """
    words = list(random.choice(list(chain.keys())))
    order = len(list(chain.keys())[0])

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state in chain:
            # Count frequencies of next words
            next_words = chain[state]
            word_counts = defaultdict(int)
            for word in next_words:
                word_counts[word] += 1

            # Apply temperature
            weights = [count ** (1.0 / temperature) for count in word_counts.values()]
            total = sum(weights)
            weights = [w/total for w in weights]

            # Choose next word based on weighted probabilities
            next_word = random.choices(list(word_counts.keys()), weights=weights)[0]
            words.append(next_word)
        else:
            break

    return ' '.join(words)

# Try different temperatures
print("\nLow temperature (more predictable):")
print(generate_text_with_temperature(chain, temperature=0.3))

print("\nHigh temperature (more random):")
print(generate_text_with_temperature(chain, temperature=2.0))


Low temperature (more predictable):
my attention yeah, these are my only intentions stay in the kitchen cookin' up, got your own bread heart full of equity, you're an asset make sure that you don't

High temperature (more random):
sponsor, nope, you're the brand now you're my rock, my colorado got that ring, just like toronto love you now, a little more tomorrow this how i feel, act like


## Challenge Tasks:

1. Implement a function to analyze the Markov chain:
   - Count the number of unique states
   - Find the most common transitions
   - Calculate the average number of possible next words for each state

In [34]:
def analyze_chain(chain):
    """
    Analyze properties of the Markov chain.

    Args:
        chain (dict): Markov chain to analyze
    """
    num_states = len(chain)
    total_transitions = sum(len(next_words) for next_words in chain.values())
    avg_transitions = total_transitions / num_states if num_states > 0 else 0

    # Find most common next word for each state
    most_common = {}
    for state, next_words in chain.items():
        word_counts = defaultdict(int)
        for word in next_words:
            word_counts[word] += 1
        most_common[state] = max(word_counts.items(), key=lambda x: x[1])

    print(f"Number of unique states: {num_states}")
    print(f"Average transitions per state: {avg_transitions:.2f}")
    print("\nMost common transitions:")
    for state, (word, count) in list(most_common.items())[:5]:  # Show top 5
        print(f"{' '.join(state)} -> {word} (count: {count})")

# Analyze our chain
print("\nChain Analysis:")
analyze_chain(chain)


Chain Analysis:
Number of unique states: 104
Average transitions per state: 1.12

Most common transitions:
picture perfect, -> you (count: 1)
perfect, you -> don't (count: 1)
you don't -> need (count: 3)
don't need -> no (count: 3)
need no -> filter (count: 1)


## Further Exploration:

Other ideas to try:
1. Modify the code to preserve punctuation
2. Add start-of-sentence and end-of-sentence tokens
3. Implement bi-directional generation
4. Create a chain that works with characters instead of words
5. Add input validation and error handling

## Adding Mood

In [35]:
# --- Mood Words ---
MOOD_WORDS = {
    "happy": {"sun", "light", "laugh", "smile", "bright", "joy"},
    "dark": {"night", "shadow", "cold", "blood", "death", "fear"},
    "calm": {"slow", "soft", "wind", "water", "quiet", "gentle"}
}

def generate_with_mood(chain, mood=None, num_words=40):
    words = list(random.choice(list(chain.keys())))
    order = len(words)

    for _ in range(num_words - order):
        state = tuple(words[-order:])
        if state not in chain:
            break
        candidates = chain[state]

        if mood:
            biased = []
            for w in candidates:
                if w.lower() in MOOD_WORDS.get(mood, set()):
                    biased.extend([w]*3)  # boost mood words
                else:
                    biased.append(w)
            next_word = random.choice(biased)
        else:
            next_word = random.choice(candidates)

        words.append(next_word)
    return " ".join(words)

# Example usage:
print("Happy text:\n", generate_with_mood(chain, mood="happy"))
print("\nDark text:\n", generate_with_mood(chain, mood="dark"))

Happy text:
 my attention yeah, these are my only intentions already passed, you don't need no approval good everywhere, don't worry 'bout no refusal second to none, you got the upper hand now don't need no mentions yeah, these are my only

Dark text:
 now, a little more tomorrow this how i feel, act like you know that you are


## Visualization

In [36]:
from collections import Counter

def print_transition_table(chain, max_states=10):
    print("State -> Possible Next Words")
    for i, (state, next_words) in enumerate(chain.items()):
        if i >= max_states:
            break
        counts = Counter(next_words)
        print(f"{state} -> {dict(counts)}")

print("\nSample Transition Table:")
print_transition_table(chain)


Sample Transition Table:
State -> Possible Next Words
('picture', 'perfect,') -> {'you': 1}
('perfect,', 'you') -> {"don't": 1}
('you', "don't") -> {'need': 3}
("don't", 'need') -> {'no': 3, 'a': 1}
('need', 'no') -> {'filter': 1, 'mentions': 1, 'approval': 1}
('no', 'filter') -> {'gorgeous,': 1}
('filter', 'gorgeous,') -> {'make': 1}
('gorgeous,', 'make') -> {"'em": 1}
('make', "'em") -> {'drop': 1}
("'em", 'drop') -> {'dead,': 1}


## Character-level markov chain

In [37]:
def build_char_chain(text, order=5):
    chain = defaultdict(list)
    for i in range(len(text) - order):
        state = tuple(text[i:i+order])
        chain[state].append(text[i+order])
    return chain

def generate_char_text(chain, num_chars=200):
    chars = list(random.choice(list(chain.keys())))
    order = len(chars)

    for _ in range(num_chars - order):
        state = tuple(chars[-order:])
        if state not in chain:
            break
        chars.append(random.choice(chain[state]))
    return ''.join(chars)

# Example usage
char_chain = build_char_chain(sample_text, order=3)
print("\nCharacter-level generation:\n", generate_char_text(char_chain, num_chars=200))


Character-level generation:
 sleeps over the lazy fox.
The lazy fox sleeps over the lazy dog jumps while the lazy fox jumps while the lazy fox sleeps over the quick brown fox sleeps while the lazy dog jumps while the lazy fox jum
